In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
import pymongo

## 📁 Define File Paths

Here we define the file paths for all the input and output files used in this workflow:

- `GFF3`: Gene annotations for the reference genome.
- `FASTA`: Reference genome sequences.
- `VCF`: Variant call format file containing genomic variants.
- `Annotated VCF`: Output path where the annotated VCF will be saved.
- `JSON`: Destination path for storing extracted genotype sequences.


In [ ]:
gff_path = "------"
fasta_path = "------"
vcf_path = "------"
annotated_vcf_path = "------"
db_path = "------"

# 1. Annotate VCF

We check whether the annotated VCF file already exists. If it doesn't, we execute an NGSEP command using the `os.system()` function. 

The VCF is annotated with:
- the reference genome (`FASTA`),
- structural gene annotations (`GFF3`),
- and stored in the `Annotated VCF` file.

Standard output and errors are redirected to a temporary log file for debugging.

In [ ]:
if not(os.path.exists(annotated_vcf_path)):
    annotate_command = rf'''ngsep -Xmx2048m VCFAnnotate \
                           -i "{vcf_path}" \
                           -r "{fasta_path}" \
                           -t "{gff_path}" \
                           -o "{annotated_vcf_path}" > /tmp/output.log 2>&1'''


    os.system(annotate_command)

## 📄 Load Annotated VCF File

The annotated VCF file is loaded into a Pandas DataFrame.

<!-- We skip the first 34 lines since they contain metadata and VCF headers not structured as tabular data. -->

In [ ]:
annotated_vcf = pd.read_csv(annotated_vcf_path, sep="\t", skiprows=34)
annotated_vcf.head(5)

## Visualize Variant Types

In this section, we extract and count the types of annotated variants using the `INFO` field in the VCF file.

- We extract the annotation code (e.g., `missense_variant`, `intron_variant`) from the `INFO` field.
- We use `numpy.unique` to count the occurrences of each type.
- Finally, we plot the distribution using a bar chart to visualize the frequency of each variant type.


In [ ]:
value, counts = np.unique(annotated_vcf['INFO'].map(lambda x : x[3:x.find(';')]), return_counts=True)
plt.bar(value, counts)
plt.xticks(rotation=90)
plt.show()

# 2. Filter relevant variants

We now focus on selecting only those variants that are biologically meaningful or likely to impact gene function.
T
his block filters the annotated VCF to retain only specific variant types that are likely to affect gene expression or protein function.
Additionally:
- A list of mutated genes is created from the filtered VCF.
- We also count the number of unique mutated genes across all individuals.

In [ ]:
relevant_variant = [ '3_prime_UTR_variant',
                     '5_prime_UTR_variant',
                     'exonic_splice_region_variant',
                     'frameshift_variant',
                     'inframe_deletion',
                     'inframe_insertion',
                     'intron_variant',
                     'missense_variant',
                     'splice_acceptor_variant',
                     'splice_donor_variant',
                     'splice_region_variant'
                     'start_lost',
                     'stop_gained',
                     'stop_lost']


filtered_annotated_vcf = annotated_vcf[annotated_vcf['INFO'].map(lambda x : x[3:x.find(';')]).isin(relevant_variant)]
filtered_annotated_vcf = filtered_annotated_vcf[filtered_annotated_vcf['INFO'].map(lambda x :  x.split(';')[1][x.split(';')[1].find('.'):] == '.1')]

mutated_genes_list = filtered_annotated_vcf['INFO'].map(lambda x : x.split(';')[1].replace('TID=','').replace('.1',''))
mutated_genes_list, variants_by_gene = np.unique(mutated_genes_list, return_counts = True)
print(f'Number of mutated genes across all the individuals: {len(mutated_genes_list)}')
filtered_annotated_vcf

# 3. Create Data Base

We connect to a local MongoDB instance and create (or access) a database named `"EG-height"`, with a collection named `"genotype"` to store the filtered variant information.

In [ ]:
db_client = pymongo.MongoClient("mongodb://localhost:11111/") 
db = db_client["EG-height"] 
collection = db["genotype"] 

# 4. Save mutated genes

This section processes each mutated gene, reconstructs its two haplotypes for each individual based on the variants present, and stores them in the MongoDB database.


We load the reference genome (FASTA) and parse it into a dictionary where each key is a chromosome ID and the value is its sequence.

We also load the GFF3 annotation file and filter to keep only rows corresponding to chromosomes (e.g., "Chr01", "Chr02", etc.).

In [ ]:
# Read fasta and GFF file
f = open(fasta_path, "r")
genome = f.read()
f.close()

# Read fasta and fasta file
chroms = genome.split('>')[1:]
chroms = {chrom[:chrom.find('\n')]: chrom[chrom.find('\n'):].replace('\n','') for chrom in chroms}

gff = pd.read_csv(gff_path, sep="\t", skiprows=1, names= ['CHROM', 'MARKER', 'TYPE', 'START', 'END', 'NULL', 'STRAND', 'NULL2', 'INFO'])
gff = gff[gff['CHROM'].map(lambda x : "Chr" in x)]

gff.head(5)

## 🧬 Extract and Store Genomic Haplotypes

For each mutated gene:

- We locate its position in the genome using the GFF annotation.
- We extract the reference gene sequence from the corresponding chromosome.
- For each individual:
  - We initialize two haplotypes (starting as the reference gene).
  - We iterate over all variants related to that gene and modify the haplotypes based on each individual’s genotype.
  - Insertions are padded to maintain consistent length using `-`.
  - Lowercase letters to represent mutated bases.

If the gene is on the negative strand (`-`), we reverse-complement the haplotypes.

Finally:
- We compute the number of mutations in each haplotype.
- Store the results (haplotypes + mutation count) in MongoDB for future analysis.

In [ ]:
translate_dict = {'A':'T', 'C':'G', 'G':'C', 'T':'A', 'a':'t', 'c':'g', 'g':'c', 't':'a', 'N':'N', '-':'-'}
inv_complement  = (lambda seq: ''.join([translate_dict[nucleotide] for nucleotide in list(seq[::-1])]))

genes_names_gff = gff['INFO'].map(lambda x: x.split(';')[0].replace('ID=',''))

for i, gene_id in enumerate(mutated_genes_list):
    gene_info = gff[genes_names_gff == gene_id]
    gene_chrom, gene_start, gene_end, gene_strand = gene_info.values[0,0], gene_info.values[0,3]-1, gene_info.values[0,4], gene_info.values[0,6]

    gene = chroms[gene_chrom][gene_start:gene_end]

    gene_variants = filtered_annotated_vcf[filtered_annotated_vcf['INFO'].map(lambda x : gene_id in x.split(';')[1])]

    individuals_ID = gene_variants.columns[9:]
    info_columns = list(gene_variants.columns[:9])

    gene_documents = []
    for individual_ID in individuals_ID:
        individual_info = gene_variants[info_columns + [individual_ID]]

        haplotype_1 = list(gene)
        haplotype_2 = list(gene)

        for variant in individual_info.values:
            variant_pos = variant[1]
            variant_ref = variant[3]
            variant_alt = variant[4].split(',')
            variant_alt = [variant.lower() for variant in variant_alt]
            
            allels = [variant_ref] + variant_alt
            insertion_size = np.max([len(allel) for allel in allels])
            allels = [''.join([allel]+['-']*(insertion_size - len(allel))) for allel in allels]

            individual_allels = variant[-1][:3].replace('.','0').split('/')

            haplotype_1[(variant_pos-1) - gene_start] = allels[int(individual_allels[0])]
            haplotype_2[(variant_pos-1) - gene_start] = allels[int(individual_allels[1])]

        haplotype_1 = ''.join(haplotype_1)
        haplotype_2 = ''.join(haplotype_2)

        if gene_strand == '-':
            haplotype_1 = inv_complement(haplotype_1)
            haplotype_2 = inv_complement(haplotype_2)
        
        n_mutations_h1 = sum(1 for char in haplotype_1 if 'a' <= char <= 'z' or char == '-')
        n_mutations_h2 = sum(1 for char in haplotype_2 if 'a' <= char <= 'z' or char == '-')

        document = {'gene_ID': gene_id,
                    'organism_ID': individual_ID,
                    'haplotype_1': haplotype_1,
                    'haplotype_2': haplotype_2,
                    'variant_counts': int(np.max([n_mutations_h1, n_mutations_h2]))}
        gene_documents.append(document)

    result = collection.insert_many(gene_documents, ordered=False)    
    print(f'Analyzed {i+1} gene of {len(mutated_genes_list)}', end='\r')

db_client.close()